In [2]:
#mainInternal


class Back(BaseException):
    """An exception to raise when a user wants to go back in a menu or when taking input
    """
    pass


class StudentRecord:
    """Objects of this class each store information of one single student"""

    def __init__(self, student_id: int, student_name: str, student_gpa: float):
        """Initialize the class with the student info given
        """
        # All the following attributes are private
        self._id: int = student_id
        self._name: str = student_name
        self._gpa: float = student_gpa
        # A dictionary with all the student info
        self._raw: dict[str, ...] = {"ID": self._id, "Name": self._name, "GPA": self._gpa}

    def id(self) -> int:
        """Return the student's id
        """
        return self._id

    def name(self) -> str:
        """Return the student's name
        """
        return self._name

    def gpa(self) -> float:
        """Returns the student's GPA
        """
        return self._gpa

    def raw(self) -> dict:
        """Returns a dictionary with ID, Name, GPA as keys associated with their respective values
        """
        return self._raw

    def modify_gpa(self, new_gpa: float):
        """Updates the students GPA to the new GPA given
        """
        self._gpa = new_gpa
        self._raw["GPA"] = new_gpa  # Update the raw dictionary


class RecordsTable:
    """Objects of this class store a variable number of StudentRecords and allows different operation on the stored
    student records such as searching, adding, removing, modifying etc."""

    def __init__(self, records_list=None):
        # Initialize private containers for data from the students
        self._ids: set[int] = set()  # A unique set of all student ids
        self._names: set[str] = set()  # A unique set of all student names
        self._records: list[StudentRecord] = []  # A list of student records (StudentRecord)
        self._raw_records: list[dict[str, ...]] = []  # A list of raw student information as dictionaries
        self._id_records: dict[int, StudentRecord] = {}  # A dictionary with ids as keys and student info as values
        self._name_records: dict[str, StudentRecord] = {}  # A dictionary with names as keys and student info as values
        self._inverse_index: dict[str, list[tuple]] = {}  # An index with single names as keys and full names as values
        # The full names are in a tuple with the first element being the full name and the second being the index of the
        # single name in the full name
        # This inverse index is only used when doing quick partial searches for student names

        # This stores the last used filename when reading or updating a file
        self.filename: str = ""

        # If a list of records was given, add them
        if records_list:
            for record in records_list:
                self.add_record(record)

    def records(self) -> list[StudentRecord]:
        """Returns a list of student info (StudentRecord)
        """
        return self._records

    def names(self) -> set[str]:
        """Returns a set of all student names
        """
        return self._names

    def ids(self) -> set[int]:
        """Returns a set of all student ids
        """
        return self._ids

    def raw(self) -> list[dict[str, ...]]:
        """Returns a list of dictionaries where each dictionary contains info for a single student in the format of:
        {"ID": student_id, "Name": student_name,"GPA": student_gpa}
        This is mainly used with show_data() in order to display the student info
        """
        return self._raw_records

    def get_record(self, student_id=None, student_name=None) -> StudentRecord:
        """Given an id or a name, returns a student if found. If not found then a KeyError is raised
        """
        try:
            if student_id is not None:
                # If an id was provided then fetch the student from the id dictionary
                return self._id_records[student_id]
            elif student_name is not None:
                # If an id was provided then fetch the student from the name dictionary
                return self._name_records[student_name]

        except KeyError as error:
            # Raise an error
            raise KeyError("Record not found") from error

        # If no id or name was supplied, raise an error
        raise KeyError("No information was supplied")

    def _add_to_inverse_index(self, full_name: str):
        """Given a full student name, adds each part of the name into the inverse index
        """
        # Split to each part
        names = full_name.split()

        # Iterate over each part
        for name_index, name in enumerate(names):
            name = name.lower()  # Set to lowercase

            # If the single name part is not present then set it
            self._inverse_index.setdefault(name, [])

            # Add to the inverse index the full name and the order of the part in the full name
            self._inverse_index[name].append((full_name, name_index))

    def _remove_from_inverse_index(self, full_name: str):
        """Removes a given full name from the inverse index
        """
        # Split to each part
        names = full_name.split()

        # Iterate over each part
        for name_index, name in enumerate(names):
            # Get all the full names for that part in the inverse index
            token_list = self._inverse_index[name.lower()]

            # Remove this specific full name
            token_list.remove((full_name, name_index))

            # If the list for the part in the index is empty after removal, delete it
            if not token_list:
                self._inverse_index.pop(name.lower())

    def clear(self):
        """Clears and deletes all student info
        """
        # Empty every single private container
        self._ids.clear()
        self._names.clear()
        self._records.clear()
        self._raw_records.clear()
        self._id_records.clear()
        self._name_records.clear()
        self._inverse_index.clear()

    def add_record(self, student_record: StudentRecord):
        """Add a student (StudentRecord)
        """
        # Add to each private container
        self._ids.add(student_record.id())
        self._names.add(student_record.name())
        self._records.append(student_record)
        self._raw_records.append(student_record.raw())
        self._id_records[student_record.id()] = student_record
        self._name_records[student_record.name()] = student_record
        self._add_to_inverse_index(student_record.name())

    def remove_record(self, student_record: StudentRecord):
        """Remove a student (StudentRecord)
        """
        # Remove from each private containers
        self._ids.remove(student_record.id())
        self._names.remove(student_record.name())
        self._records.remove(student_record)
        self._raw_records.remove(student_record.raw())
        self._id_records.pop(student_record.id())
        self._name_records.pop(student_record.name())
        self._remove_from_inverse_index(student_record.name())

    def search_record(self, query: str):
        """Returns a new RecordsTable with all the students that match the search query
        """
        query = query.strip()
        query_names = query.split()
        maximum_differences = len(query) // 3

        if len(query_names) > 1:
            sorted_names = levenshtein_automaton(query, sorted(self.names()), maximum_differences)

        else:
            possible_names = levenshtein_automaton(query, sorted(self._inverse_index.keys()), maximum_differences)
            name_results = {full_name: order
                            for name in possible_names
                            for full_name, order in self._inverse_index[name.lower()]}
            sorted_names = *sorted(name_results, key=name_results.get),

        return RecordsTable([self.get_record(student_name=name) for name in sorted_names])

    def read_file(self, filename: str):
        """Reads the file given if found into the records
        """
        self.filename = filename  # Update the current filename

        # Open in read mode
        with open(filename, "r") as file:
            # Read the first line
            line = file.readline().strip()

            # Loop until the end of the file (line becomes empty)
            while line:
                # Split the line
                data = line.split(",")
                # The first, second, and third index in the split line should the ID, Name, GPA respectively
                # Create a new StudentRecord with the info
                student_record = StudentRecord(student_id=int(data[0]), student_name=data[1].strip(),
                                               student_gpa=float(data[2]))

                # Add it to self
                self.add_record(student_record)

                # Read the next line
                line = file.readline().strip()

    def update_file(self, filename=None):
        """Updates the file if given, otherwise updates the file given when making the object for the first time
        """
        # If no filename was given then default to the filename in self
        if not filename:
            filename = self.filename

        # Open file in write mode
        with open(filename, "w") as file:
            # Iterate over every record
            for student_record in self._records:
                # Write a line in the correct format for each student:
                # ID, Name, GPA
                file.write(f"{student_record.id()}, {student_record.name()}, {student_record.gpa()}\n")

    def search_analyzer(self, response: str):
        possible_records = self.search_record(response)

        if not possible_records.records():
            print("No results")
            raise Back

        # Show data
        show_data(possible_records.raw())

        choice_number = menu(["Choose"], back_option="Search Again")

        if choice_number == 1:  # Option Choose
            return input("Enter ID: ")

    def present_id_check(self, response: str) -> tuple[bool, str]:
        try:
            student_id = int(response)

        except ValueError:
            return False, "the ID should be 9 integer numbers"

        if len(response) != 9:
            return False, "the ID should be 9 integer numbers"

        if student_id in self._ids:
            return True, ""

        else:
            return False, "the ID is not there"

    def new_id_check(self, response: str) -> tuple[bool, str]:
        try:
            student_id = int(response)

        except ValueError:
            return False, "the ID should be 9 integer numbers"

        if len(response) != 9:
            return False, "the ID should be 9 integer numbers"

        if student_id not in self._ids:
            return True, ""

        else:
            return False, "the ID is already there"


class Inputs:

    def __init__(self):
        self._prompts = []
        self._inputs = {}

    def add_prompt(self, prompt_text: str, check, analyzer=None):
        self._inputs[prompt_text] = (analyzer, check)
        self._prompts.append(prompt_text)

    def take_inputs(self) -> ...:
        print("Type 'cancel' to stop taking input or 'back' to undo.")

        results = [None] * len(self._prompts)

        current_prompt_index = 0
        while current_prompt_index < len(self._prompts):
            prompt = self._prompts[current_prompt_index]
            response = input(prompt)

            if response.lower() == "cancel" or (current_prompt_index == 0 and response.lower() == "back"):
                raise Back

            elif response.lower() == "back":
                current_prompt_index -= 1
                continue

            input_analyzer, input_check = self._inputs[prompt]
            if input_analyzer:
                try:
                    analyzed_response = input_analyzer(response)
                except Back:
                    continue

            else:
                analyzed_response = response

            if input_check is None:
                results.pop(current_prompt_index)
                results.insert(current_prompt_index, analyzed_response)
                current_prompt_index += 1
                continue

            check_success, check_message = input_check(analyzed_response)
            if check_success:
                results.pop(current_prompt_index)
                results.insert(current_prompt_index, analyzed_response)
                current_prompt_index += 1

            else:
                print(check_message)

        if len(results) > 1:
            return tuple(results)
        else:
            return results[0]


def menu(options: list[str], back_option: str = "Back",
         prompt="Enter a number to choose or type in part of the option: ") -> int:
    """Displays the options given and prompts the user.
    Returns an integer indicating the option selected starting from 1
    Raise Back (error) if the user chooses the back option

    Give the options as a list of strings that will be displayed
    """
    # Prepare the options as a table to be shown using view_data()
    display_data = []
    for option_number, option in enumerate(options, start=1):
        display_data.append({"Number": option_number, "Option": option})

    # Add the back option
    display_data.append({"Number": 0, "Option": back_option})
    options = [back_option] + options

    # Display the data
    show_data(display_data)

    # Loop until a valid option is chosen
    while True:
        # Take a response from the user
        response = input(prompt).strip()

        if response.isdigit():
            # If the response is a number, check if the number is present
            choice_number = int(response)
            if choice_number == 0:  # If the user chooses back, raise Back
                print("Chose Option", back_option)
                raise Back

            elif 0 < choice_number < len(options):  # If the user chose a valid number, return the choice
                print(f"Chose Option {choice_number}: {options[choice_number]}")
                return choice_number

            else:  # Otherwise, notify the user and retake input
                print("Please enter a number that is present.")

        else:
            # If the response is not a number, search for it within the options
            possible_options = levenshtein_automaton(response, sorted(options), 0)
            if possible_options:  # If there are possible options
                # Take the most possible option
                option = possible_options[0]

                # Check where the option lies in the options list
                choice_number = options.index(option)

                if choice_number == 0:  # If the user chooses back, raise Back
                    print("Chose Option", back_option)
                    raise Back

                else:  # If the option is
                    print(f"Chose Option {choice_number}: {option}")
                    return choice_number

            else:  # Otherwise, notify the user and retake input
                print("Please type in a valid choice.")


def show_data(data: list[dict[str, ...]]):
    """Prints data formatted as a list of dictionaries, where each dictionary in the list is a row and each
    key in the dictionary a column.

    *Note: Each dictionary should have identical keys.

    Example input and output:

    >records = [
    {'ID': 123456789, 'Name': 'mohammed khalifa', 'Gpa': 2.25},
    {'ID': 202312340, 'Name': 'khalid ahmed', 'Gpa': 3.5},
    {'ID': 202345771, 'Name': 'Mohammad Abdu', 'Gpa': 0.0}
    ]
    >show_data(records)
    _______________________________________
    | ID        | Name             | GPA  |
    _______________________________________
    | 123456789 | mohammed khalifa | 2.25 |
    | 202312340 | khalid ahmed     | 3.50 |
    | 202345771 | Mohammad Abdu    | 0.00 |
    _______________________________________
    """
    column_titles = []
    columns_max_width = {}

    if not data:
        print("There is no data to show\n\n")
        return

    # Get the column titles for the data to display
    for title in data[0]:
        column_titles.append(title)
        columns_max_width[title] = len(title)

    # Get the maximum width of each column in the data
    for row in data:
        for title in column_titles:
            if len(str(row[title])) > columns_max_width[title]:
                columns_max_width[title] = len(str(row[title]))

    # Get the max row length in the table
    max_line_length = 1 + sum(list(columns_max_width.values())) + 3 * len(column_titles)
    row_boundary = max_line_length * "_"

    # Print the header
    print(row_boundary)
    for title in column_titles:
        print(f"| {title}{' ' * (columns_max_width[title] - len(title))} ", end="")
    print("|")

    # Print the table rows
    print(row_boundary)
    for row in data:  # rows
        for column in column_titles:
            print(f"| {row[column]}{' ' * (columns_max_width[column] - len(str(row[column])))} ", end="")
        print("|")

    # Close the table
    print(row_boundary)
    print()


def valid_name_check(response: str) -> tuple[bool, str]:
    if len(response.split()) >= 2:
        return True, ""
    else:
        return False, "Please enter at least a first and second name."


def valid_gpa_check(response: str) -> tuple[bool, str]:
    try:
        student_gpa = float(response)

    except ValueError:
        return False, "The GPA should be a number between 0 and 4"

    if 0 <= student_gpa <= 4:
        return True, ""
    else:
        return False, "The GPA should be a number between 0 and 4"


def valid_filename_check(response: str) -> tuple[bool, str]:
    parts = response.split(".")  # Split the file
    for part in parts:
        if not part.isalnum():
            return False, "Please enter only alphanumeric characters for the filename."

    return True, ''


def create_file(filename):
    """Creates a file with the name filename if not present.
    Goes back if an error occurs.
    """
    try:
        # Open filename with mode 'w' in order to create the file if it is not present
        with open(filename, mode="w"):
            pass   # Do nothing

    except IOError:
        # In case an error occurs, go back
        print(f"Could not create '{filename}'")
        raise Back

    else:
        # If the file was successfully created, notify the user
        print(f"Successfully created/opened file '{filename}'")
        print()


def read_file_into_record(records: RecordsTable, filename: str = ""):
    """Reads file contents and updates records with the data in it. If the file could not be opened or something wrong
    happens, prompts the user with options.
    Supports going back
    """
    # Define an input for taking in filenames from the user in case the current file cannot be opened for any reason
    new_file_name_input = Inputs()
    new_file_name_input.add_prompt("Enter a new filename: ", valid_filename_check)

    # Take a filename if no filename was provided
    if not filename:
        filename = new_file_name_input.take_inputs()

    # Read from file until successful while handling any errors
    while True:
        try:
            # Attempt to read file
            records.read_file(filename=filename)

            # If no error occurs, print message and exit
            print(f"Successfully opened and read from file '{filename}'")
            print()
            return

        except FileNotFoundError:
            # If the file is not found, notify user
            print(f"ERROR: Could not find file '{filename}'")
            print()

            # Prompt the user to either choose to create a new file or to set a new file to read from
            print("Do you want to?")
            choice_number = menu([f"Create File '{filename}'", "Set New Filename"])
            print("\n")

            if choice_number == 1:  # Option Create File
                create_file(filename)

            else:  # Option Set New Filename
                # Set a new filename to attempt again
                filename = new_file_name_input.take_inputs()

        except IndexError:
            # If the file is not formatted correctly, notify user and take a new file
            print(f"ERROR: '{filename}' File format is incorrect")
            print()
            # Set a new filename to attempt again
            filename = new_file_name_input.take_inputs()

        except IOError:
            # Catch any other error and notify user
            print(f"ERROR: Unexpected error occurred while trying to read '{filename}'")
            print()
            # Set a new filename to attempt again
            filename = new_file_name_input.take_inputs()


def update_file_from_record(records: RecordsTable, filename):
    """Opens file and updates its data with the records. If the file could not be opened, prompts the user
    for a new file. If the file is present, checks if there is data to be read from the file. If so, warns the user
    that the file will be overwritten then prompts the user.
    Supports going back
    """
    # Define an input for taking in filenames from the user in case the current file cannot be opened for any reason
    new_file_name_input = Inputs()
    new_file_name_input.add_prompt("Enter a new filename: ", valid_filename_check)
    # Attempt to update file until successful
    while True:
        try:
            # Attempt updating
            records.update_file(filename)
            # If successful then exit
            print(f"Successfully wrote to file '{filename}'")
            print()
            return

        except IOError:
            # If any file error occurs, prompt the user for a new filename
            print(f"ERROR: Could not write to '{filename}'")
            filename = new_file_name_input.take_inputs()


def write_to_new_file(records: RecordsTable):
    inputs = Inputs()
    inputs.add_prompt("Enter the new file name:", valid_filename_check)

    file_name = inputs.take_inputs()
    update_file_from_record(records, file_name)


def switch_file(records: RecordsTable):
    records.clear()
    inputs = Inputs()
    inputs.add_prompt("Enter the new file name:", valid_filename_check)

    file_name = inputs.take_inputs()
    read_file_into_record(records, file_name)


In [3]:
#editRecords


def add_record(student_records: RecordsTable):
    """Prompts the user for new student info and then adds it to the given records table
    Supports going back
    """

    # Define an input for taking student info
    inputs = Inputs()
    inputs.add_prompt("Enter ID:", student_records.new_id_check)
    inputs.add_prompt("Enter Name:", valid_name_check)
    inputs.add_prompt("Enter GPA:", valid_gpa_check)

    # Take the inputs
    input_return = inputs.take_inputs()

    # Extract the user's response
    student_id, student_name, student_gpa = int(input_return[0]), input_return[1], round(float(input_return[2]), 2)

    # Create a new StudentRecord with the info the user gave
    new_record = StudentRecord(student_id, student_name, student_gpa)

    # Add it to the records table
    student_records.add_record(new_record)


def remove_record(student_records: RecordsTable):
    """Prompts the user for a student to delete from the given records table and removes it if found
    Supports going back
    """
    # Loop until the user goes back or the modification is successful
    while True:
        # Show the records table so that the user can see the current student info
        show_data(student_records.raw())

        # Allow the user to choose the method of choosing a student to remove
        print("Choose a student to modify:")
        choice_number = menu(["By Searching For a Student",
                              "By ID"])

        # Each method takes different input prompts
        if choice_number == 1:  # Option Searching for Student
            # Define an input for searching for a student to remove
            inputs = Inputs()
            inputs.add_prompt("Search:", student_records.present_id_check, analyzer=student_records.search_analyzer)

        else:  # Option By ID:
            # Define an input for taking a present student id
            inputs = Inputs()
            inputs.add_prompt("Enter ID:", student_records.present_id_check)

        try:
            # Take the input from the user and convert into an integer
            student_id = int(inputs.take_inputs())
        except Back:
            # If the user goes back here, return to the choice menu at the beginning
            continue  # Retry

        # Get the record from the records table
        record_to_remove = student_records.get_record(student_id=student_id)

        # Remove the record from the records table
        student_records.remove_record(record_to_remove)
        
        print("Successfully removed student.")
        # Exit
        return


def modify_record_menu(student_records: RecordsTable):
    """Prompts user for a student to modify the gpa of
    Supports going back
    """
    # Loop until the user goes back or the modification is successful
    while True:
        # Show the records table so that the user can see the current student info
        show_data(student_records.raw())

        # Allow the user to choose the method of choosing a student
        print("Choose a student to modify:")
        choice_number = menu(["By Searching For a Student",
                              "By ID"])

        # Each method takes different input prompts
        if choice_number == 1:  # Option Searching for Student
            # Define an input for searching for a student then taking a new valid gpa to modify to
            inputs = Inputs()
            inputs.add_prompt("Search:", student_records.present_id_check, analyzer=student_records.search_analyzer)
            inputs.add_prompt("Enter GPA:", valid_gpa_check)

        else:  # Option By ID:
            # Define an input for taking a present student id and a new valid gpa to modify to
            inputs = Inputs()
            inputs.add_prompt("Enter ID:", student_records.present_id_check)
            inputs.add_prompt("Enter GPA:", valid_gpa_check)

        try:
            # Take the input from the user
            input_response = inputs.take_inputs()
        except Back:
            # If the user goes back here, return to the choice menu at the beginning
            continue  # Retry

        # Get the id and gpa from the user's response
        student_id, student_gpa = int(input_response[0]), round(float(input_response[1]), 2)

        # Get the StudentRecord from the records table using the id the user provided
        record_to_modify = student_records.get_record(student_id=student_id)

        # Modify to StudentRecord's GPA to the new value the user provided
        record_to_modify.modify_gpa(student_gpa)
        
        print("Successfully modified student.")
        # Exit
        return


In [4]:
#sortRecord


def sort_menu(student_records: RecordsTable):
    """Prompts the user to select a sort type and order and then displays the results
    Supports going back
    """
    # Loop until the user goes back or a successful sort occurs
    while True:
        # Prompt the user for a sort type
        print("Choose Sort Type:")
        sort_type_number = menu(["Sort by ID", "Sort by GPA", "Sort by Name"])
        print("\n")

        try:
            # Prompt the user for a sort order
            print("Choose Sort Order:")
            sort_order_number = menu(["Ascending", "Descending"])
            print("\n")

        except Back:
            # If the user goings back here, return to the first menu
            continue  # Retry menu selection

        # Define a boolean depending on the sort order response from the menu
        if sort_order_number == 1:  # Option Ascending
            descending = False

        else:  # Option Descending
            descending = True

        # Sort
        if sort_type_number == 1:  # Sort by ID
            # noinspection PyTypeChecker
            sorted_student_records = sorted(student_records.records(), key=StudentRecord.id, reverse=descending)

        elif sort_type_number == 2:
            # noinspection PyTypeChecker
            sorted_student_records = sorted(student_records.records(), key=StudentRecord.gpa, reverse=descending)

        else:
            # noinspection PyTypeChecker
            sorted_student_records = sorted(student_records.records(), key=StudentRecord.name, reverse=descending)

        # Create a new records table with the sorted records
        sorted_record_table = RecordsTable(sorted_student_records)

        # Display the sorted results
        show_data(sorted_record_table.raw())

        # Exit
        return


In [5]:
# searchInternal

def calculate_next_levenshtein_row(character1: str, main_string: str, previous_row: list) -> list[int]:
    """Calculates the next levenshtein row according to the levenshtein algorithm.
    A character from a string, the main string to compare it against, and the previous row need to be provided.
    """
    # The column number is the first number in the previous row
    starting_column = previous_row[0] + 1
    # Set the first number in the new row to the previous row's column number + 1
    next_row: list = [starting_column, ]

    # Loop over every character in the main string while comparing it the character1
    for column_number, character2 in enumerate(main_string, start=1):
        # Apply the levenshtein algorithm to calculate the next element in the row
        next_row.append(min(
            next_row[column_number - 1] + 1,
            previous_row[column_number] + 1,
            previous_row[column_number - 1] + (character1 != character2)))

    # Return the next row
    return next_row


def levenshtein_automaton(comparison_string: str, sorted_strings, threshold: int, case_sensitive=False) -> tuple[str]:
    """Basically a search function.
    This is an optimized algorithm that takes a list of strings and compares them against a
    comparison string. It discards any strings that are less similar than a given threshold.
    The list of strings should be sorted for maximum speed.
    """
    # Set the comparison string to lowercase if not case-sensitive
    comparison_string = comparison_string.lower() if not case_sensitive else comparison_string

    # Initialise the results dictionary
    results = {}

    # A list to store previously calculated levenshtein rows
    automatons = []

    # Initialise the previous string to use when optimizing
    prev_str = ''

    # Loop over every string in the sorted strings and compare them against the comparison string
    # Add each string to results that is similar to the comparison string
    for original_string in sorted_strings:
        # Set the string to lowercase if not case-sensitive
        string = original_string.lower() if not case_sensitive else original_string

        # Start from the first index of the string
        character_index = 0

        # The following loop is an optimization to skip any characters in the string that were already
        # previously calculated
        # It sets the edits to a previously calculated levenshtein row in the automatons list
        # Loop if the there is a previous string and if the letters are identical in both the prev and current string
        while prev_str and prev_str[character_index] == string[character_index]:
            # Move to the next character
            character_index += 1

            # If the next character is not identical or the end of either string is reached, break out of loop
            if prev_str[character_index] != string[character_index] or character_index == len(string) or\
                    character_index == len(automatons):
                # Set the current edits to the previous character
                edits = automatons[character_index - 1]
                break

        else:
            # If no similarity between the current string and the previous string is present, or there is no prev string
            # then initialise edits to a default starting levenshtein row
            edits = list(range(len(comparison_string) + 1))

        # Cut out the levenshtein rows that are no longer useful
        automatons = automatons[:character_index]

        # Update the previous string for the next iteration of the loop
        prev_str = string

        # If the minimum amount of edits possible is greater than the threshold, the loop exits
        # and the word is not appended into the results
        # If the string ends, the loop exits and the word is not appended unless the total amount of edits is less than
        # the threshold
        while min(edits) <= threshold and character_index < len(string) and character_index < len(comparison_string):
            # If the comparison strings ends and the threshold is not exceeded, break and add the word to the
            # results
            edits = calculate_next_levenshtein_row(string[character_index], comparison_string, edits)

            # Add the calculated row to automatons for possible optimization
            automatons.append(edits)

            # Move to next character
            character_index += 1

        # The final index in edits is the total number of edits required
        if edits[-1] > threshold:
            # If the total number of edits required for the current string to be equal to comparison string is greater
            # than the threshold, skip to next string
            continue

        # Add the word to results with the minimum possible amount of edits (similarity)
        results[original_string] = min(edits)

    # Return a tuple of the results sorted by the edit number (Lower edits means more similar)
    return *sorted(results, key=results.get),


In [6]:
#searchFromRecords


def search_menu(record_table: RecordsTable):
    """Prompts the user for a search type then searches the record table and displays the results
    Supports going back
    """
    # Loop until the user goes back or a successful search is made
    while True:
        # Prompt the user to choose a search type
        print("Choose Search Type:")
        choice_number = menu(["Search by Name", "Search by ID"])
        print("\n")

        try:
            # Run each search type
            if choice_number == 1:  # Option Search by Name
                search_by_name(record_table)

            else:  # Option Search by ID
                search_by_id(record_table)

            # Exit
            return

        except Back:
            # If any of the options went back, display the search type menu again
            pass


def search_by_name(record_table: RecordsTable):
    """Takes a query from the user and searches the record table and displays the search results
    Supports going back
    """
    # Define an input for taking a search query
    inputs = Inputs()
    inputs.add_prompt("Search: ", None)

    # Take the query
    query = inputs.take_inputs()

    # Search the records table
    results_records = record_table.search_record(query)

    # Show the results
    show_data(results_records.raw())


def search_by_id(record_table: RecordsTable):
    """Takes an id from user and displays the info of the student with that id
    Supports going back
    """
    # Define an input for taking an id that is present in the records table
    inputs = Inputs()
    inputs.add_prompt("Enter ID: ", record_table.present_id_check)

    # Take the input
    student_id = int(inputs.take_inputs())

    # Get the record from the table
    student_record = record_table.get_record(student_id=student_id)

    # Show the record
    show_data([student_record.raw()])



In [7]:
#mergeFile


def merge_records(current_student_records: RecordsTable):
    """Prompts the user for a new filename, reads the student data, and then merges it with
    the current data. If any conflicts occur while merging, solves conflict according to the
    behaviour the user chooses.
    Supports going back.
    """
    # Read a new file into records
    new_student_records = RecordsTable()
    read_file_into_record(new_student_records)

    # Take behaviour type of merge from user
    print("Choose what to do when encountering info conflicts:")
    behaviour_type = menu(["Ask before merging",
                           "Keep old",
                           "Overwrite old with new"])
    print()

    # Merge new file records into current student records
    for new_record in new_student_records.records():
        try:
            old_record = current_student_records.get_record(student_id=new_record.id())

        except KeyError:
            # The record is not present
            # Add it to the current records table
            current_student_records.add_record(new_record)
            continue  # Move on to the next record

        # If an old record was found, compare its contents to the new record and merge according to behaviour type
        merged_name = old_record.name()
        merged_gpa = old_record.gpa()
        merge = False
        try:
            if old_record.name() != new_record.name():
                if behaviour_type == 1:  # Ask user
                    print("A conflict in names has been detected for id:", old_record.id())
                    print("Old name in current file:", old_record.name())
                    print("New name in new file:", new_record.name())
                    print()
                    print("Choose which name to keep for", old_record.id())
                    choice = menu([old_record.name(),
                                   new_record.name()], back_option="Skip Record")

                    if choice == 1:  # Keep old name
                        merged_name = old_record.name()

                    else:  # Set new name
                        merged_name = new_record.name()
                        merge = True

                elif behaviour_type == 2:  # Keep old
                    merged_name = old_record.name()

                else:  # Overwrite old with new
                    merged_name = new_record.name()
                    merge = True

            if old_record.gpa() != new_record.gpa():
                if behaviour_type == 1:  # Ask user
                    print(f"A conflict in gpa has been detected for {merged_name} with id:", old_record.id())
                    print("Old gpa in current file:", old_record.gpa())
                    print("New gpa in new file:", new_record.gpa())
                    print()
                    print(f"Choose which gpa to keep for {merged_name}")
                    choice = menu([str(old_record.gpa()),
                                   str(new_record.gpa())], back_option="Skip Record")

                    if choice == 1:  # Keep old name
                        merged_gpa = old_record.gpa()

                    else:  # Set new name
                        merged_gpa = new_record.gpa()
                        merge = True

                elif behaviour_type == 2:  # Keep old
                    merged_gpa = old_record.gpa()

                else:  # Overwrite old with new
                    merged_gpa = new_record.gpa()
                    merge = True

            # If any merge was required, create a new record with the merged contents and add it to the table

            if merge:
                merged_record = StudentRecord(old_record.id(), merged_name, merged_gpa)

                # Delete old record
                current_student_records.remove_record(old_record)

                # Add new merged record
                current_student_records.add_record(merged_record)

            else:
                # Otherwise don't add anything
                pass

        except Back:
            # If the user goes back, skip adding the record
            print("Skipping", new_record.id())

    print("Finished merging from file:", new_student_records.filename)


In [8]:


def calculate_average(student_records: RecordsTable):
    try:
        gpa_sum = 0
        for student in student_records.records():
            gpa_sum += student.gpa()
 
        average = gpa_sum / len(student_records.records())

        print("The average is :", average)

    except ZeroDivisionError:
        print("There is no data to calculate the average.")


def top_performing_students(records: RecordsTable):
    sorted_student_records = sorted(records.records(), key=StudentRecord.gpa, reverse=True)

    counter = 0

    top_students = []
    top_gpas = []

    for student in sorted_student_records:

        if counter == 3:
            if student.gpa() in top_gpas:
                counter -= 1
            else:
                break

        top_students.append(student)
        top_gpas.append(student.gpa())
        counter += 1

    show_data(RecordsTable(top_students).raw())


In [9]:
#main


# Define the default file to read from
DEFAULT_STUDENT_FILE_NAME = "students2.txt"


def main():
    """The main program
    """
    # Initialise a new RecordsTable where all the student records are
    records = RecordsTable()

    # Attempt to read from the default file
    try:
        read_file_into_record(records, DEFAULT_STUDENT_FILE_NAME)

    except Back:
        # If the user goes back, exit the program
        print("Exiting program prematurely without doing anything")
        return

    # Define the menu options
    menu_options = ["View Records",
                    "Add Record",
                    "Remove Record",
                    "Modify Record",
                    "Search Records",
                    "Sort Records",
                    "Top Performing Students",
                    "Calculate Average",
                    "Save to Current File",
                    "Switch File",
                    "Write to File",
                    "Merge Files"]

    # Loop and display the main menu while responding to user input
    while True:
        print("Choose: ")

        try:  # Display the menu
            selected_option = menu(options=menu_options, back_option="Save and Exit")

        except Back:
            break  # If the user goes back, exit the main menu loop

        print("\n")
        # Run the function the user chose from the menu, while ignoring if the option goes back
        try:
            if selected_option == 1:  # Option View Records
                show_data(records.raw())

            elif selected_option == 2:  # Option Add Record
                add_record(records)

            elif selected_option == 3:  # Option Remove Record
                remove_record(records)

            elif selected_option == 4:  # Option Modify Record
                modify_record_menu(records)

            elif selected_option == 5:  # Option Search Record
                search_menu(records)

            elif selected_option == 6:  # Option Sort Records
                sort_menu(records)

            elif selected_option == 7:  # Option Top Performing
                top_performing_students(records)

            elif selected_option == 8:  # Option Average
                calculate_average(records)

            elif selected_option == 9:  # Option Save to Current File
                update_file_from_record(records, records.filename)

            elif selected_option == 10:  # Option Switch File
                switch_file(records)

            elif selected_option == 11:  # Option Write to File
                write_to_new_file(records)

            elif selected_option == 12:  # Option Merge Files
                merge_records(records)

            input("Press ENTER to continue")

        except Back:  # If any of these options go back, ignore it and continue loop
            pass

    # After exiting the main menu, update the current student file
    try:
        update_file_from_record(records, records.filename)

    except Back:
        print("Exiting program without saving.")
        return

    print("Program Closed")


In [10]:
main()

Successfully opened and read from file 'students2.txt'

Choose: 
____________________________________
| Number | Option                  |
____________________________________
| 1      | View Records            |
| 2      | Add Record              |
| 3      | Remove Record           |
| 4      | Modify Record           |
| 5      | Search Records          |
| 6      | Sort Records            |
| 7      | Top Performing Students |
| 8      | Calculate Average       |
| 9      | Save to Current File    |
| 10     | Switch File             |
| 11     | Write to File           |
| 12     | Merge Files             |
| 0      | Save and Exit           |
____________________________________

Enter a number to choose or type in part of the option: 1
Chose Option 1: View Records


_______________________________________
| ID        | Name             | GPA  |
_______________________________________
| 123456789 | mohammed khalifa | 2.25 |
| 202312340 | khalid ahmed     | 3.7  |
| 202345770 | Ha